## **Урок 14. Transfer learning**

## Задание

1. взять данные из
https://www.kaggle.com/datasets/mrapplexz/bashim-quotes
обучить модель GPT для генерации своих цитат

2. взять новостные данные из
https://github.com/natasha/corus
load_lenta2
нам понадобиться сам текст и заголовок
обучить модель T5/ или GPT для генерации заголовков для статей

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 57.3 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
     |████████████████████████████████| 596 kB 58.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.2 MB/s 
     |████████████████████████████████| 140 kB 69.2 MB/s 
     |████████████████████████████████| 1.1 MB 59.6 MB/s 
     |████████████████████████████████| 212 kB 68.1 MB/s 
     |████████████████████████████████| 127 kB 72.2 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 271 kB 57.9 MB/s 
     |████████████████████████████████| 144 kB 74.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import json
import torch

Генерируем  цитаты по примерам случайных записей из датасета https://www.kaggle.com/datasets/mrapplexz/bashim-quotes

In [4]:
DATASET_PATH = '/content/dataset.jsonl'

with open(DATASET_PATH) as f:
     df = pd.read_json(DATASET_PATH, lines=True).set_index('id')
df.head(1)

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."


In [5]:
df.drop(['date', 'rating'], axis=1, inplace=True)
df.head()

,text
id,
1,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,<томатик_рад> а ты не чувствуешь красоту мира?...
3,"<Дор> ""мышка, почему у тебя такие большие глаз..."
4,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ..."
5,<Ohtori_Akio> мы - как разработчики - живём с ...


In [6]:
# for i in df['text']:
#     print(i)

text=df['text']

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [8]:
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
#model_name = 'Grossmend/rudialogpt3_medium_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

In [9]:
import random

In [10]:
sep = '\n***\n'

prefix = sep.join([''] + random.sample(list(text), k=5) + [''])

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]
print(prefix)


***
<IsillorN> вдуплился = поселился в дупле на ПМЖ
<walek> а что тогда такое отдуплился?
<awayko> выселился из дупла в связи с невозможностью платить по кредиту за него
***
ххх: а то вас послушать, так есть можно только чёрный хлеб и запивать водой.
ууу: Да ващет и этого нельзя - в хлебе дрожжи, а в воде микробы.
zzz: Алармо! Веган в чате!
***
Nightcrawler 
ты не знаешь как в максе ввести ограничение на наследственное вращение по двум осям

[IHC]SpiriT 
нет, не знаю

Nightcrawler 
заметь, это был не вопрос
***
xxx: Насчет трудного детства линуха
xxx: Вы думаете это на ровном месте хейт?
xxx: Я то че, мне все равно
xxx: Я еще с RT-11 и DOS-ом работал в свое время
xxx: А вот уволюсь я
xxx: И бедные престарелые тетеньки познают
xxx: Сколько надо секаса, чтобы просто подключить сетевой диск
xxx: Это вам не щелкнул, ввел логин и пароль, галочку сохранить поставил и все завелось
xxx: Это изучи пол операционной системы, блин
xxx: Файловую систему, mount, fstab, chmod
xxx: И потом еще трахай

In [11]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+128, 
    repetition_penalty=4.2, 
    temperature=0.7,
    num_beams=10,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


yuri_loskutov: Как вы относитесь к тому, что если у человека нет высшего образования, он может стать прокурором?
***
sergej_pozharsky: В чем разница между юристом и адвокатом? Юристу нужно знать законы, а адвокату - их толкование.
***
dmitry_gordon: Что делать, если тебя обвиняют в том, чего ты не совершал?
***
timofey_fedorov: Если человек говорит "я невиновен", значит ли это, что ему нечего бояться?
***



Впринципе источник на основании которого были сгенерированы цитаты вполне узноваем. Сгенерированные фразы имеют смысл.